In [29]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 155000
pd.options.display.max_columns = 500
pd.options.display.float_format = '{:20,.2f}'.format
seed = np.random.seed(66)

In [30]:
# !pip install openpyxl

In [31]:
data = pd.read_excel("customer_data.xlsx")

In [32]:
data.columns.values

array(['Client_ID', 'Age_Category', 'State', 'Annual_Income',
       'Source_of_Income', 'Investment_Amount', 'Dependents',
       'Expected_ROI'], dtype=object)

In [33]:
print(data.dtypes)

Client_ID             int64
Age_Category         object
State                object
Annual_Income         int64
Source_of_Income     object
Investment_Amount     int64
Dependents            int64
Expected_ROI         object
dtype: object


In [34]:
data['Age_Category'] = data['Age_Category'].astype(str)

In [35]:
# data['AGE_BUCKET'] = pd.cut(data.Age_Category, [0,30,40,50,60,999] , labels = [1, 2, 3, 4, 5])
# Define a function to map Age_Category to numerical categories
def map_age_category(age_str):
    if age_str == '<30':
        return 1
    elif age_str == '30-40':
        return 2
    elif age_str == '40-50':
        return 3
    elif age_str == '50-60':
        return 4
    elif age_str == '>60':
        return 5
    else:
        return None  # Handle unexpected values

# Apply the mapping function to the Age_Category column
data['AGE_BUCKET'] = data['Age_Category'].apply(map_age_category)


In [36]:
list = []
for i in data['Expected_ROI']:
    if i == '<10%':
        list.append(4)
    elif i == '10-15%':
        list.append(3)  
    elif i == '15-25%':
        list.append(2)   
    else:
        list.append(1)    
          
data['TARGET_BUCKET'] = list      

In [37]:
for i in data['AGE_BUCKET']:
    if i==1:
            data['Max_Investment'] = 0.6 * data['Annual_Income']
    elif i==2:
             data['Max_Investment'] = 0.5 * data['Annual_Income']
    elif i==3:
             data['Max_Investment'] = 0.3 * data['Annual_Income']
    elif i==4:
            data['Max_Investment'] = 0.2 * data['Annual_Income']
    else:
            data['Max_Investment'] = 0.1 * data['Annual_Income']

In [38]:
data.head()

,Client_ID,Age_Category,State,Annual_Income,Source_of_Income,Investment_Amount,Dependents,Expected_ROI,AGE_BUCKET,TARGET_BUCKET,Max_Investment
0,1,50-60,Karnataka,3085317,Corporate employee,625000,2,15-25%,4,2,"925,595.10"
1,2,>60,Haryana,3859127,self employed,2013889,2,>25%,5,1,"1,157,738.10"
2,3,30-40,Maharashtra,2351190,Business,446429,3,15-25%,2,2,"705,357.00"
3,4,40-50,Kerala,2192460,Corporate employee,505952,3,15-25%,3,2,"657,738.00"
4,5,40-50,Kerala,2192460,Corporate employee,505952,3,15-25%,3,2,"657,738.00"


In [39]:
data['Inv_Ratio'] = data['Max_Investment']/data['Investment_Amount']

In [40]:
data['Risk_Score_N'] = 1 - ((data['Dependents']*3 + data['Inv_Ratio']*4
                             + data['AGE_BUCKET']*3)*data['TARGET_BUCKET'])/100

In [41]:
print(data['Risk_Score_N'])

0                   0.52
1                   0.77
2                   0.57
3                   0.54
4                   0.54
5                   0.54
6                   0.52
7                   0.57
Name: Risk_Score_N, dtype: float64


In [42]:
list = []
for i in abs(data['Risk_Score_N']):
    if i <= 0.2:
        list.append('Conservative')
    elif i <= 0.4:
        list.append('Moderatively Conservative')  
    elif i <= 0.6:
        list.append('Moderatively Aggresive')    
    elif i <= 0.8:
        list.append('Aggresive')   
    else:
        list.append('Very Aggresive')    
          
data['Risk Profile'] = list      

In [43]:
data['Risk Profile'].value_counts()

Risk Profile
Moderatively Aggresive    7
Aggresive                 1
Name: count, dtype: int64

In [44]:
data.head()

,Client_ID,Age_Category,State,Annual_Income,Source_of_Income,Investment_Amount,Dependents,Expected_ROI,AGE_BUCKET,TARGET_BUCKET,Max_Investment,Inv_Ratio,Risk_Score_N,Risk Profile
0,1,50-60,Karnataka,3085317,Corporate employee,625000,2,15-25%,4,2,"925,595.10",1.48,0.52,Moderatively Aggresive
1,2,>60,Haryana,3859127,self employed,2013889,2,>25%,5,1,"1,157,738.10",0.57,0.77,Aggresive
2,3,30-40,Maharashtra,2351190,Business,446429,3,15-25%,2,2,"705,357.00",1.58,0.57,Moderatively Aggresive
3,4,40-50,Kerala,2192460,Corporate employee,505952,3,15-25%,3,2,"657,738.00",1.30,0.54,Moderatively Aggresive
4,5,40-50,Kerala,2192460,Corporate employee,505952,3,15-25%,3,2,"657,738.00",1.30,0.54,Moderatively Aggresive


In [45]:
# data[data['Risk Profile'] == 'Very Aggresive']

In [46]:
data['Amt_Investment'] = data[['Investment_Amount','Max_Investment']].min(axis=1)

In [47]:
data.shape

(8, 15)

In [48]:
# def get_company_list(dat1,dat2,col1,col2):
    
#     list = []
    
#     for i in dat1[col1]:
#         dat_1 = dat1[dat1[col1] == i]
#         for j in dat_1[col2]:
#             dat_2 = dat2[dat2[col2] == j][['Name', 'BSE Code', 'NSE Code', 'Industry', 'Current Price','Company_Rank']].sort_values("Company_Rank").head(10)
#             dat_2 = dat_2.sample(frac = 0.5)
#             dat_2['Client Id'] = dat_1['Client Id'] 
#             dat_2['Customer Name'] = dat_1['Name'] 
#             dat_2['Amt_Investment'] = dat_1['Amt_Investment'] 
#             dat_2 = dat_2[['Client Id','Customer Name','Amt_Investment','Name','BSE Code', 'NSE Code', 'Industry', 'Current Price']]
#             return dat_2 
# # pd.concat(dat1[['Client Id','Name','Amt_Investment']],dat_2)   

In [49]:
def get_company_list(dat1,dat2,col1,col2):
    
    list = []
    
    for i in dat1[col1]:
        print(i)
        dat_1 = dat1[dat1[col1] == i]
        print(dat_1['Client_ID'])
        for j in dat_1[col2]:
#             print(j)
            dat_2 = dat2[dat2[col2] == j][[ 'Name','BSE Code', 'NSE Code', 'Industry', 'Current Price','Risk Profile','Company_Rank']].sort_values("Company_Rank").head(10)
            dat_2 = dat_2.sample(frac = 0.5).reset_index(drop = True)
#             dat_2['Client Id'] = dat_1['Client Id'] 
#             dat_2['Customer Name'] = dat_1['Name'] 
#             dat_2['Amt_Investment'] = dat_1['Amt_Investment'] 
#             dat_2 = dat_2[['Client Id','Customer Name','Amt_Investment','Name','BSE Code', 'NSE Code', 'Industry', 'Current Price']]
#         dat_2['Client Id'] = dat_1['Client Id'] 
#         dat_2['Customer Name'] = dat_1['Name'] 
#         dat_2['Amt_Investment'] = dat_1['Amt_Investment'] 
#         dat_2 = dat_2[['Client Id','Customer Name','Amt_Investment','Name','BSE Code', 'NSE Code', 'Industry', 'Current Price']]
            
        return dat_2 

In [50]:
data_comp = pd.read_csv("Data_Companies.csv")

In [51]:
results = get_company_list(data,data_comp,'Client_ID','Risk Profile').sort_values("Company_Rank").head(10)

1
0    1
Name: Client_ID, dtype: int64


In [52]:
results

,Name,BSE Code,NSE Code,Industry,Current Price,Risk Profile,Company_Rank
0,Oracle Fin.Serv.,532466,OFSS,Computers - Software - Medium / Small,"3,075.10",Moderatively Aggresive,1.00
3,Axis Bank,532215,AXISBANK,Banks - Private Sector,859.20,Moderatively Aggresive,2.00
4,APL Apollo Tubes,533758,APLAPOLLO,Steel - Large,"1,081.70",Moderatively Aggresive,7.00
1,Bharti Airtel,532454,BHARTIARTL,Telecommunications - Service Provider,837.30,Moderatively Aggresive,9.00
2,Divi's Lab.,532488,DIVISLAB,Pharmaceuticals - Indian - Bulk Drugs,"3,287.35",Moderatively Aggresive,10.00
